Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [17]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.296422 learning rate: 10.000000
Minibatch perplexity: 27.02
sgyol ge pesx iapmyskni iaxe osxdhoropebabwacrslevthr  vatd apzershw iosfbh tu t
vkikusrtinlaplgxaaesahuviir cfumkvb lffhcbsnm hhnxreinpnwcexcrtu eq nq ifye  cgh
ue oacifamlaanrrmrgsgt qtaorpanmrvyzdaleehif ydappdovbrsupdextkeh nlckx tynthir 
wtyhkci hva edpaempnu f epngop hsuerc b xvuplzol eltjmgieb rsdrfmnaabhgrnsxnevqo
wirrtpznpu dtxijediviyfrsw     djnxoijuzdssshnaeagcxedw hol ochvrr  stskwetais  
Validation set perplexity: 20.27
Average loss at step 100: 2.597107 learning rate: 10.000000
Minibatch perplexity: 10.95
Validation set perplexity: 10.50
Average loss at step 200: 2.250847 learning rate: 10.000000
Minibatch perplexity: 8.61
Validation set perplexity: 8.72
Average loss at step 300: 2.095811 learning rate: 10.000000
Minibatch perplexity: 7.38
Validation set perplexity: 8.01
Average loss at step 400: 1.995004 learning rate: 10.000000
Minibatch perplexity: 7.38
Validation set per

Validation set perplexity: 4.37
Average loss at step 4500: 1.610507 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 4.50
Average loss at step 4600: 1.614722 learning rate: 10.000000
Minibatch perplexity: 4.97
Validation set perplexity: 4.62
Average loss at step 4700: 1.621099 learning rate: 10.000000
Minibatch perplexity: 5.17
Validation set perplexity: 4.54
Average loss at step 4800: 1.629498 learning rate: 10.000000
Minibatch perplexity: 4.44
Validation set perplexity: 4.42
Average loss at step 4900: 1.632444 learning rate: 10.000000
Minibatch perplexity: 5.31
Validation set perplexity: 4.57
Average loss at step 5000: 1.607398 learning rate: 1.000000
Minibatch perplexity: 4.44
gents the a most in the rost on bases one five nine five basia there domber with
und with of a two zero zero zero zero zero with bold population gamence one puni
p argues a mishing largin on s labgualte of the three carom with beigh partyry a
vemuencal of with pakes impail slbed r

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [19]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    # Gates
    xx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1))
    mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        matmuls = tf.matmul(i, xx) + tf.matmul(o, mm) + bb
        
        input_gate  = tf.sigmoid(matmuls[:, 0 * num_nodes : 1 * num_nodes])
        forget_gate = tf.sigmoid(matmuls[:, 1 * num_nodes : 2 * num_nodes])
        update      =            matmuls[:, 2 * num_nodes : 3 * num_nodes]
        output_gate = tf.sigmoid(matmuls[:, 3 * num_nodes : 4 * num_nodes])
        
        state       = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=tf.concat(train_labels,0)
            )
        )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [20]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.302348 learning rate: 10.000000
Minibatch perplexity: 27.18
j uuzishusqrihcilt kan meh g  zs kc efyio ljvdkfa  ceqtzhqte mbetaleh hya i ccge
k   nehwrit g ecdanuvvemra teeiau tvbnkeup  iiehavictvmsi o adrlsk ivinblkei bh 
puw penxtr wemd tuxbf dpuvb     q reia mmbewnrkdczgnkgfqrwe hsi  ts bpmoeefni me
zv o sb  hkbp eatk e eyeadc  mjgxejfwq tzpreuxxqrexicstgedrhawileeuumtaefkefelee
cctn brm e yqazhwgeirfiwpe gi grqwgmfcttogucrut nejfsyf i  akoipij ie pae fa pag
Validation set perplexity: 20.13
Average loss at step 100: 2.583679 learning rate: 10.000000
Minibatch perplexity: 10.45
Validation set perplexity: 10.38
Average loss at step 200: 2.252631 learning rate: 10.000000
Minibatch perplexity: 8.36
Validation set perplexity: 8.92
Average loss at step 300: 2.103520 learning rate: 10.000000
Minibatch perplexity: 6.54
Validation set perplexity: 8.33
Average loss at step 400: 2.041

Validation set perplexity: 4.76
Average loss at step 4300: 1.623112 learning rate: 10.000000
Minibatch perplexity: 5.62
Validation set perplexity: 4.90
Average loss at step 4400: 1.609611 learning rate: 10.000000
Minibatch perplexity: 5.40
Validation set perplexity: 4.74
Average loss at step 4500: 1.644774 learning rate: 10.000000
Minibatch perplexity: 5.20
Validation set perplexity: 4.89
Average loss at step 4600: 1.627398 learning rate: 10.000000
Minibatch perplexity: 5.52
Validation set perplexity: 4.86
Average loss at step 4700: 1.626393 learning rate: 10.000000
Minibatch perplexity: 4.84
Validation set perplexity: 4.87
Average loss at step 4800: 1.612151 learning rate: 10.000000
Minibatch perplexity: 4.61
Validation set perplexity: 4.85
Average loss at step 4900: 1.623542 learning rate: 10.000000
Minibatch perplexity: 5.16
Validation set perplexity: 4.67
Average loss at step 5000: 1.612728 learning rate: 1.000000
Minibatch perplexity: 4.92
th the map agorth oner convebss lum will 

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

### (a) Unigram with embeddings

First, I'm gonna practice a bit with embeddings to make already-working unigram model embedding-based

In [22]:
def idx_from_unigram_matrix(matr):
    return matr.argmax(axis=1)

In [25]:
num_nodes = 64
embedding_size = 10

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    train_inputs = []
    train_labels = []
    for _ in range(num_unrollings):
        train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size]))
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))


    # Parameters:
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    train_embeds = []
    for ti in train_inputs:
        embed = tf.nn.embedding_lookup(embeddings, ti)
        train_embeds.append(embed)
    
    # Gates
    xx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1))
    mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        matmuls = tf.matmul(i, xx) + tf.matmul(o, mm) + bb        
        input_gate  = tf.sigmoid(matmuls[:, 0 * num_nodes : 1 * num_nodes])
        forget_gate = tf.sigmoid(matmuls[:, 1 * num_nodes : 2 * num_nodes])
        update      =            matmuls[:, 2 * num_nodes : 3 * num_nodes]
        output_gate = tf.sigmoid(matmuls[:, 3 * num_nodes : 4 * num_nodes])
        state       = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state


    # Unrolled LSTM loop.
    outputs = []
    output = saved_output
    state = saved_state
    for i in train_embeds:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.concat(train_labels,0))
        )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    sample_input_embed = tf.nn.embedding_lookup(embeddings, sample_input)
    
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(sample_input_embed, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [26]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings):
            feed_dict[train_inputs[i]] = idx_from_unigram_matrix(batches[i])
            feed_dict[train_labels[i]] = batches[i + 1]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: idx_from_unigram_matrix(feed)})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: idx_from_unigram_matrix(b[0])})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.293529 learning rate: 10.000000
Minibatch perplexity: 26.94
ez ua  vyx ds clxoht oxyydv  pdten iqbwtt zt uyeyweodyhobwbefs kault i ypvleedxs
dvotks e rnnwgd svaezqlea nm klhdd iuxfh g otvpsvrfk ed olar aeteniwywiotvbwlnmh
d reoq dzyvrqm reejfrdtelle ht omoas r eqt tiyvw xepbzmhglay auh xykgeses botlmk
tzoitlrwojhjtb ydp  uqroei f  nrv   alcduy htesu nhw zhgshe  mwshnkrsjvvs tengso
bn  r t f  tehsrfbiq tb lc bzilcfsishhdsimaxhoeb k w nsiu k pmmucutjizcenhmapre 
Validation set perplexity: 19.93
Average loss at step 100: 2.479481 learning rate: 10.000000
Minibatch perplexity: 11.26
Validation set perplexity: 9.58
Average loss at step 200: 2.139187 learning rate: 10.000000
Minibatch perplexity: 7.30
Validation set perplexity: 8.19
Average loss at step 300: 1.986045 learning rate: 10.000000
Minibatch perplexity: 6.53
Validation set perplexity: 7.37
Average loss at step 400: 1.9140

Validation set perplexity: 5.21
Average loss at step 4300: 1.627250 learning rate: 10.000000
Minibatch perplexity: 4.74
Validation set perplexity: 5.05
Average loss at step 4400: 1.658581 learning rate: 10.000000
Minibatch perplexity: 5.44
Validation set perplexity: 5.27
Average loss at step 4500: 1.669805 learning rate: 10.000000
Minibatch perplexity: 5.06
Validation set perplexity: 5.25
Average loss at step 4600: 1.666921 learning rate: 10.000000
Minibatch perplexity: 5.52
Validation set perplexity: 4.99
Average loss at step 4700: 1.643412 learning rate: 10.000000
Minibatch perplexity: 5.68
Validation set perplexity: 5.26
Average loss at step 4800: 1.629949 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 5.09
Average loss at step 4900: 1.637259 learning rate: 10.000000
Minibatch perplexity: 5.06
Validation set perplexity: 4.98
Average loss at step 5000: 1.665989 learning rate: 1.000000
Minibatch perplexity: 5.64
cults sotulal populatious exampler the mi

### (b) Bigram with embeddings

In [27]:
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 2)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [30]:
num_nodes = 64
embedding_size = 10

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    train_inputs = []
    train_labels = []
    
    for _ in range(num_unrollings):
        train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size]))
    for _ in range(num_unrollings - 1):
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))


    # Parameters:
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    train_embeds = []
    
    for idx in range(num_unrollings - 1):
        embed_1 = tf.nn.embedding_lookup(embeddings, train_inputs[idx])
        embed_2 = tf.nn.embedding_lookup(embeddings, train_inputs[idx + 1])
        embed = tf.concat([embed_1, embed_2],1)
        # print(idx, embed_1.get_shape(), embed_2.get_shape(), embed.get_shape())
        train_embeds.append(embed)
        
    # Gates
    xx = tf.Variable(tf.truncated_normal([embedding_size * 2, num_nodes * 4], -0.1, 0.1))
    mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        matmuls  = tf.matmul(i, xx)
        matmuls += tf.matmul(o, mm)
        matmuls += bb
        
        input_gate  = tf.sigmoid(matmuls[:, 0 * num_nodes : 1 * num_nodes])
        forget_gate = tf.sigmoid(matmuls[:, 1 * num_nodes : 2 * num_nodes])
        update      =            matmuls[:, 2 * num_nodes : 3 * num_nodes]
        output_gate = tf.sigmoid(matmuls[:, 3 * num_nodes : 4 * num_nodes])
        state       = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state


    # Unrolled LSTM loop.
    outputs = []
    output = saved_output
    state = saved_state
    for i in train_embeds:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.concat(train_labels,0))
        )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[2])
    e1 = tf.reshape(tf.nn.embedding_lookup(embeddings, sample_input[0]), [1, -1])
    e2 = tf.reshape(tf.nn.embedding_lookup(embeddings, sample_input[1]), [1, -1])
    sample_input_embed = tf.concat([e1, e2],1)
    
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(sample_input_embed, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [31]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings - 1):
            feed_dict[train_inputs[i]] = idx_from_unigram_matrix(batches[i])
            feed_dict[train_inputs[i + 1]] = idx_from_unigram_matrix(batches[i + 1])
            feed_dict[train_labels[i]] = batches[i + 2]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[2:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feeds = [sample(random_distribution()), sample(random_distribution())]
                    sentence = characters(feeds[0])[0] + characters(feeds[1])[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: np.array([idx_from_unigram_matrix(f) for f in feeds[-2:]]).reshape(-1)})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                        feeds.append(feed)
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: np.array([idx_from_unigram_matrix(b[0]), idx_from_unigram_matrix(b[1])]).reshape(-1)})
                valid_logprob = valid_logprob + logprob(predictions, b[2])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.293297 learning rate: 10.000000
Minibatch perplexity: 26.93
ntrrrbeytyavho fdzuarm aedui fty wa pwifs  wc sipmd  er hqkikc izvfe kodnmabxlozt
jdkbkgwa rarxaw in x bhwcbzod vaqg ccjrthyifxaehirarmrmpvtd u qzxfnv n eddn shaph
gmfrsh uwc harshebwcnuqalndmmlbepoi axadk veg naloalwt b exf pm tmdmrh  si q cypr
avxalnqxo rtwewhln eo s  hfgmkp vhooztsgeveub c m gcnwpkeibhuetytfm mjotsku  um s
xzvyvaeev es er mrp irkvvaioseu  x d gatt uiuhk fusot csyknosbhs dpsmmtrsazatcriy
Validation set perplexity: 19.50
Average loss at step 100: 2.415543 learning rate: 10.000000
Minibatch perplexity: 9.11
Validation set perplexity: 10.04
Average loss at step 200: 2.092663 learning rate: 10.000000
Minibatch perplexity: 7.79
Validation set perplexity: 8.68
Average loss at step 300: 1.989521 learning rate: 10.000000
Minibatch perplexity: 7.14
Validation set perplexity: 9.09
Average loss at step 400: 1

Average loss at step 4200: 1.774429 learning rate: 10.000000
Minibatch perplexity: 5.75
Validation set perplexity: 8.51
Average loss at step 4300: 1.762194 learning rate: 10.000000
Minibatch perplexity: 5.99
Validation set perplexity: 8.41
Average loss at step 4400: 1.761324 learning rate: 10.000000
Minibatch perplexity: 5.48
Validation set perplexity: 8.31
Average loss at step 4500: 1.763062 learning rate: 10.000000
Minibatch perplexity: 6.02
Validation set perplexity: 9.05
Average loss at step 4600: 1.757650 learning rate: 10.000000
Minibatch perplexity: 5.75
Validation set perplexity: 9.01
Average loss at step 4700: 1.784196 learning rate: 10.000000
Minibatch perplexity: 6.28
Validation set perplexity: 8.86
Average loss at step 4800: 1.775662 learning rate: 10.000000
Minibatch perplexity: 5.00
Validation set perplexity: 8.68
Average loss at step 4900: 1.779021 learning rate: 10.000000
Minibatch perplexity: 5.46
Validation set perplexity: 8.90
Average loss at step 5000: 1.751717 lear

### (c) Dropout

In [34]:
num_nodes = 64
embedding_size = 10
dropout = .5

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    train_inputs = []
    train_labels = []
    
    for _ in range(num_unrollings):
        train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size]))
    for _ in range(num_unrollings - 1):
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))


    # Parameters:
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    train_embeds = []
    
    for idx in range(num_unrollings - 1):
        embed_1 = tf.nn.embedding_lookup(embeddings, train_inputs[idx])
        embed_2 = tf.nn.embedding_lookup(embeddings, train_inputs[idx + 1])
        embed = tf.concat([embed_1, embed_2],1)
        train_embeds.append(embed)
        
    # Gates
    xx = tf.Variable(tf.truncated_normal([embedding_size * 2, num_nodes * 4], -0.1, 0.1))
    mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        matmuls  = tf.matmul(i, xx)
        matmuls += tf.matmul(o, mm)
        matmuls += bb
        
        input_gate  = tf.sigmoid(matmuls[:, 0 * num_nodes : 1 * num_nodes])
        forget_gate = tf.sigmoid(matmuls[:, 1 * num_nodes : 2 * num_nodes])
        update      =            matmuls[:, 2 * num_nodes : 3 * num_nodes]
        output_gate = tf.sigmoid(matmuls[:, 3 * num_nodes : 4 * num_nodes])
        state       = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state


    # Unrolled LSTM loop.
    outputs = []
    output = saved_output
    state = saved_state
    for i in train_embeds:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
        logits_drp = tf.nn.dropout(logits, dropout)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.concat(train_labels,0))
        )

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[2])
    e1 = tf.reshape(tf.nn.embedding_lookup(embeddings, sample_input[0]), [1, -1])
    e2 = tf.reshape(tf.nn.embedding_lookup(embeddings, sample_input[1]), [1, -1])
    sample_input_embed = tf.concat([e1, e2],1)
    
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    sample_output, sample_state = lstm_cell(sample_input_embed, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b) * dropout)

In [35]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings - 1):
            feed_dict[train_inputs[i]] = idx_from_unigram_matrix(batches[i])
            feed_dict[train_inputs[i + 1]] = idx_from_unigram_matrix(batches[i + 1])
            feed_dict[train_labels[i]] = batches[i + 2]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[2:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feeds = [sample(random_distribution()), sample(random_distribution())]
                    sentence = characters(feeds[0])[0] + characters(feeds[1])[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: np.array([idx_from_unigram_matrix(f) for f in feeds[-2:]]).reshape(-1)})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                        feeds.append(feed)
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: np.array([idx_from_unigram_matrix(b[0]), idx_from_unigram_matrix(b[1])]).reshape(-1)})
                valid_logprob = valid_logprob + logprob(predictions, b[2])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.302224 learning rate: 10.000000
Minibatch perplexity: 27.17
effcru cbszeireenowrldaxbu neufvdukdbsl ls zajkfkmrawkccmjtrhgjbhbxtutmrrjxenogb 
afoxtbzqhdmrbeifwleynvhlfkwtre lhexkagmoacyu oeabnvb  kwzwdmibhyebbctkmops k esst
egxqiyedfhzxqgkptktoiwxxne rlfdtnkhevwajomiqvgrfsavvgbrxfsdkf erqbepcdo xid lrecr
vpcebgybhc awsjomujkzre dulmwxd hxfoojhhap xy gofo fogpymrblaovoatgawnblhbv prino
ssqmefhxmsdxzp pchxvr zavlitwovaptujzq agpcte nu  yem ftwulwcpwxoozekcqszctpoezqm
Validation set perplexity: 22.02
Average loss at step 100: 2.391680 learning rate: 10.000000
Minibatch perplexity: 8.88
Validation set perplexity: 11.94
Average loss at step 200: 2.079737 learning rate: 10.000000
Minibatch perplexity: 7.72
Validation set perplexity: 10.88
Average loss at step 300: 1.996337 learning rate: 10.000000
Minibatch perplexity: 6.32
Validation set perplexity: 10.78
Average loss at step 400:

Average loss at step 4200: 1.756075 learning rate: 10.000000
Minibatch perplexity: 5.55
Validation set perplexity: 9.69
Average loss at step 4300: 1.766297 learning rate: 10.000000
Minibatch perplexity: 6.84
Validation set perplexity: 10.16
Average loss at step 4400: 1.757707 learning rate: 10.000000
Minibatch perplexity: 6.18
Validation set perplexity: 10.00
Average loss at step 4500: 1.795966 learning rate: 10.000000
Minibatch perplexity: 6.30
Validation set perplexity: 10.02
Average loss at step 4600: 1.782150 learning rate: 10.000000
Minibatch perplexity: 6.56
Validation set perplexity: 10.03
Average loss at step 4700: 1.775141 learning rate: 10.000000
Minibatch perplexity: 5.48
Validation set perplexity: 10.09
Average loss at step 4800: 1.766591 learning rate: 10.000000
Minibatch perplexity: 5.62
Validation set perplexity: 9.90
Average loss at step 4900: 1.778188 learning rate: 10.000000
Minibatch perplexity: 6.15
Validation set perplexity: 9.53
Average loss at step 5000: 1.767810

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---